
Óscar Fernández Sánchez

Sergio Orejón Pérez

Jordy Ivan San Martín Ponce

Carlos Garrido Junco

In [ ]:
#Para knn
import sklearn.datasets as datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

#Para bayes
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
#Con gauss seria: 
from sklearn.naive_bayes import GaussianNB


#Para arboles 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier


#Matriz de confusion 
from sklearn.metrics import classification_report, confusion_matrix
#Para la validación cruzada KFold
from sklearn import datasets, metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

BASE DATOS  DE HEARTCARE

In [ ]:
#Para añadir la base de datos - está incluido el csv en la entrega
from google.colab import files
import io

uploaded= files.upload()
heart_attack = pd.read_csv(io.BytesIO(uploaded['heart_UCI.csv']))
X = heart_attack.loc[:,heart_attack.columns != 'target']
Y = heart_attack['target'].values 





Saving heart_UCI.csv to heart_UCI.csv


In [ ]:
#Al final usamos la misma base de datos de la práctica anterior
#Comprobación que le hicimos a la base de datos que probamos y nos daba muchos errores, la hemos añadido como parte de nuestro aprendizaje.
for i in range(len(X.age)):
  for j in X.columns:
    if(X.loc[i,j]=='?'):
      X.loc[i,j]=np.nan
X.isna().sum()

## como en esa base de datos faltaban datos pues usamos KNN_Imputer para rellenar las ? que habia. En la base de datos usada actualmente no hace falta
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=10)
knn_imputer_fit = knn_imputer.fit(X.values)
imputed_data = knn_imputer_fit.transform(X)
print(imputed_data)
X=pd.DataFrame(imputed_data,columns=X.columns)
X.isna().sum()

[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. ...  0.  0.  2.]
 [41.  0.  1. ...  2.  0.  2.]
 ...
 [68.  1.  0. ...  1.  2.  3.]
 [57.  1.  0. ...  1.  1.  3.]
 [57.  0.  1. ...  1.  1.  2.]]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
dtype: int64

DIVIDIR LOS CONJUNTOS EN ENTRENAMIENTO TEST  Y VALIDACIÓN 

In [ ]:
# Separar dataset en sets de entrenamiento y test
X = heart_attack.loc[:,heart_attack.columns != 'target']
Y = heart_attack['target'].values 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
X_train, X_val, Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=0.111) # Con estos valores lo dividimos en 80 entrenamiento 20 test 20 validacion
print(X_train)
print("*******************")
print(X_test)
print("*******************")
print(X_val)
print("+++++++++++++++++++++++++++++++++++++")
print(Y_train)
print("*******************")
print(Y_test)
print("*******************")
print(Y_val)

     age  sex  cp  trestbps  chol  ...  exang  oldpeak  slope  ca  thal
154   39    0   2       138   220  ...      0      0.0      1   0     2
221   55    1   0       140   217  ...      1      5.6      0   0     3
260   66    0   0       178   228  ...      1      1.0      1   2     3
233   64    1   0       120   246  ...      1      2.2      0   1     2
44    39    1   2       140   321  ...      0      0.0      2   0     2
..   ...  ...  ..       ...   ...  ...    ...      ...    ...  ..   ...
73    51    1   0       140   261  ...      1      0.0      2   0     2
96    62    0   0       140   394  ...      0      1.2      1   0     2
176   60    1   0       117   230  ...      1      1.4      2   2     3
2     41    0   1       130   204  ...      0      1.4      2   0     2
164   38    1   2       138   175  ...      0      0.0      2   4     2

[241 rows x 13 columns]
*******************
     age  sex  cp  trestbps  chol  ...  exang  oldpeak  slope  ca  thal
168   63    1   0  

KNN

In [ ]:
#Para KNN

# Elegir numero de vecinos
clf=KNeighborsClassifier(n_neighbors=10)

# Ajustar clf al set de entrenamiento
clf.fit(X_train,Y_train)

#Predicciones en set de test
predictions=clf.predict(X_test)

#Sacamos los valores de la predicción y de la matriz de confusión
accuracy_train=clf.score(X_train,Y_train)
accuracy_test=clf.score(X_test,Y_test)
print("Knn")
print("Antes de optimizar el hiperparametro")
print ("Precision set entrenamiento: {:.2f}".format(accuracy_train))
print ("Precision set de test: {:.2f}".format(accuracy_test))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))
#Aplicar algoritmo Cross Validation
neighbors = [] 
cv_scores = [] 
print("prueba kfold")

scores = cross_val_score(clf, X_train, Y_train, cv=kf, scoring="accuracy")

print("Metricas cross_validation", scores)

print("Media de cross_validation", scores.mean())

#Con este algoritmo sacamos el numero de vecinos optimos para nuestro problema
from sklearn.model_selection import cross_val_score 
for k in range(1, 51, 2): #no puede ser numero par
    neighbors.append(k) 
    knn = KNeighborsClassifier(n_neighbors = k) 
    scores = cross_val_score(knn, X_train, Y_train, cv = 10, scoring = 'accuracy') 
    cv_scores.append(scores.mean()) 
MSE = [1-x for x in cv_scores]   
optimal_k = neighbors[MSE.index(min(MSE))] 
print('El numero optimo de vecinos es:  % d ' % optimal_k) 

#Rehaciendo algoritmo con valor de vecinos optimizado
print("Rehacer el modelo KNN con el hiperparametro optimizado")

clf=KNeighborsClassifier(n_neighbors=optimal_k)
clf.fit(X_train,Y_train)
predictions=clf.predict(X_test)
#Sacamos los valores de la predicción y de la matriz de confusión
accuracy_train=clf.score(X_train,Y_train)
accuracy_test=clf.score(X_test,Y_test)
print ("Precision set de entrenamiento: {:.2f}".format(accuracy_train))
print ("Precision set de test:: {:.2f}".format(accuracy_test))
print ("#####################################################")
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))
print("prueba kfold")

scores = cross_val_score(clf, X_train, Y_train, cv=kf, scoring="accuracy")

print("Metricas cross_validation", scores)

print("Media de cross_validation", scores.mean())



Knn
Antes de optimizar el hiperparametro
Precision set entrenamiento: 0.67
Precision set de test: 0.74
[[ 9  1]
 [ 7 14]]
              precision    recall  f1-score   support

           0       0.56      0.90      0.69        10
           1       0.93      0.67      0.78        21

    accuracy                           0.74        31
   macro avg       0.75      0.78      0.74        31
weighted avg       0.81      0.74      0.75        31

prueba kfold
Metricas cross_validation [0.63265306 0.64583333 0.5        0.64583333 0.5       ]
Media de cross_validation 0.5848639455782314
El numero optimo de vecinos es:   3 
Rehacer el modelo KNN con el hiperparametro optimizado
Precision set de entrenamiento: 0.80
Precision set de test:: 0.74
#####################################################
[[ 8  2]
 [ 6 15]]
              precision    recall  f1-score   support

           0       0.57      0.80      0.67        10
           1       0.88      0.71      0.79        21

    accuracy   

ARBOLES

In [ ]:
model =  DecisionTreeClassifier(criterion="entropy", max_depth=3)
model.fit(X_train, Y_train)
print(model.get_depth())
#Sacamos los valores de la predicción y de la matriz de confusión
Y_ab_pred_train=model.predict(X_train)
Y_ab_pred_test=model.predict(X_test)
print("Arbol de clasificacion")
print("Porcentaje de predicciones correctas (training): ",100*metrics.accuracy_score(Y_train,Y_ab_pred_train))
print("Porcentaje de predicciones correctas (test): ",100*metrics.accuracy_score(Y_test,Y_ab_pred_test))
print ("#####################################################")

print(confusion_matrix(Y_test,Y_ab_pred_test))
print(classification_report(Y_test,Y_ab_pred_test))

#Algoritmo de validación cruzada con el que podemos ver nuestros resultados
print("prueba kfold")


scores = cross_val_score(model, X_test, Y_test, cv=kf, scoring="accuracy")

print("Metricas cross_validation test ", scores)

print("Media de cross_validation test ", scores.mean())

3
Arbol de clasificacion
Porcentaje de predicciones correctas (training):  80.91286307053942
Porcentaje de predicciones correctas (test):  93.54838709677419
#####################################################
[[ 8  2]
 [ 0 21]]
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.91      1.00      0.95        21

    accuracy                           0.94        31
   macro avg       0.96      0.90      0.92        31
weighted avg       0.94      0.94      0.93        31

prueba kfold
Metricas cross_validation test  [0.85714286 1.         1.         0.83333333 0.83333333]
Media de cross_validation test  0.9047619047619048


BAYES

In [ ]:
#Para Bayes

#Usamos el modelo Gausiano
model=GaussianNB(var_smoothing=1e-9)
#Pero podriamos usar el modelo Multinomial
#model=MultinomialNB()
model.fit(X_train, Y_train)

#Sacamos los valores de la predicción y de la matriz de confusión
Y_pred_train=model.predict(X_train)
Y_pred_test=model.predict(X_test)
print("Bayes")
print("Porcentaje de predicciones correctas (training): ",100*metrics.accuracy_score(Y_train,Y_pred_train))
print("Porcentaje de predicciones correctas (test): ",100*metrics.accuracy_score(Y_test,Y_pred_test))
print(confusion_matrix(Y_test,Y_pred_test))
print(classification_report(Y_test,Y_pred_test))

#Algoritmo de validación cruzada con el que podemos ver nuestros resultados
print("prueba kfold")

scores = cross_val_score(model, X_train, Y_train, cv=kf, scoring="accuracy")

print("Metricas cross_validation", scores)

print("Media de cross_validation", scores.mean())

Bayes
Porcentaje de predicciones correctas (training):  81.74273858921161
Porcentaje de predicciones correctas (test):  90.32258064516128
[[ 9  1]
 [ 2 19]]
              precision    recall  f1-score   support

           0       0.82      0.90      0.86        10
           1       0.95      0.90      0.93        21

    accuracy                           0.90        31
   macro avg       0.88      0.90      0.89        31
weighted avg       0.91      0.90      0.90        31

prueba kfold
Metricas cross_validation [0.91836735 0.8125     0.79166667 0.70833333 0.75      ]
Media de cross_validation 0.7961734693877551


REDES NEURONALES

In [ ]:
#Creamos la red neuronal 
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(25,25,25), max_iter=1500,solver='lbfgs',activation='relu')
mlp.fit(X_train, Y_train)
predictions = mlp.predict(X_test)
print(predictions)
#Sacamos los resultados de la predicción y de la matriz de confusión
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

#Utilizamos la validación cruzada para comprobar nuestros resultados
print("prueba kfold")

scores = cross_val_score(mlp, X_train, Y_train, cv=kf, scoring="accuracy")

print("Metricas cross_validation", scores)

print("Media de cross_validation", scores.mean())

[0 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1]
[[ 9  1]
 [ 3 18]]
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.95      0.86      0.90        21

    accuracy                           0.87        31
   macro avg       0.85      0.88      0.86        31
weighted avg       0.88      0.87      0.87        31

prueba kfold
Metricas cross_validation [0.83673469 0.89583333 0.8125     0.66666667 0.72916667]
Media de cross_validation 0.7881802721088434


*Prueba*  con el conjunto de validacion


In [ ]:
#Prueba de la pregunta 6 a la red neuronal con los datos del set de validación 
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(25,25,25), max_iter=1500,solver='lbfgs',activation='relu')
mlp.fit(X_train, Y_train)
predictions = mlp.predict(X_val)
print(predictions)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_val,predictions))
print(classification_report(Y_val,predictions))

print("prueba kfold")

scores = cross_val_score(model, X_val, Y_val, cv=kf, scoring="accuracy")

print("Metricas cross_validation", scores)

print("Media de cross_validation", scores.mean())

[0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0]
[[14  3]
 [ 3 11]]
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        17
           1       0.79      0.79      0.79        14

    accuracy                           0.81        31
   macro avg       0.80      0.80      0.80        31
weighted avg       0.81      0.81      0.81        31

prueba kfold
Metricas cross_validation [0.71428571 0.66666667 0.83333333 1.         0.66666667]
Media de cross_validation 0.7761904761904762


Perceptron

In [ ]:
#Perceptrón utilizado para la pregunta 7 aplicando el modelo lineal 
from sklearn.linear_model import Perceptron
p = Perceptron()
p.fit(X_train, Y_train)

predictions = p.predict(X_test)

print("*********")
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

*********
[[ 3 10]
 [ 0 18]]
              precision    recall  f1-score   support

           0       1.00      0.23      0.38        13
           1       0.64      1.00      0.78        18

    accuracy                           0.68        31
   macro avg       0.82      0.62      0.58        31
weighted avg       0.79      0.68      0.61        31

